**Image preparation**

Firstly all libaries will be imported.

In [21]:
import cv2 as cv
import os
import numpy as np
import random

Define input and output folders for the processed and cropped images so the images can be prepared for the model

In [ ]:
input_folders = ["Raw/Plaster", "Raw/Machine", "Raw/Philips", "Raw/Torx"]
output_folders = ["Threshholds/Plaster", "Threshholds/Machine", "Threshholds/Philips", "Threshholds/Torx"]
dataset_train = ["Dataset/train/Plaster", "Dataset/train/Machine", "Dataset/train/Philips", "Dataset/train/Torx"]
dataset_val = ["Dataset/val/Plaster", "Dataset/val/Machine", "Dataset/val/Philips", "Dataset/val/Torx"]

for folder in output_folders + dataset_train + dataset_val:
    os.makedirs(folder, exist_ok=True)

When all the output folders are created, the kernel used for blob dilation will be defined. Here it is important to use a odd-number for dilation size.

In [ ]:
kernel = np.ones((9,9), np.uint8)

With the dilation kernel defined, each image will be processed using grayscale, gaussian blue, dilation and will be cropped from the original image.

In [ ]:
for input_folder, output_folder, dataset_train, dataset_val in zip(input_folders, output_folders, dataset_train, dataset_val):
    for filename in os.listdir(input_folder):
        if filename.endswith((".jpg")):

            # Read the image from the dataset
            img_path = os.path.join(input_folder, filename)
            img = cv.imread(img_path)

            # Convert the image to grayscale
            gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

            # Apply Gaussian blur before thresholding
            blurred = cv.GaussianBlur(gray, (5, 5), 0)

            # Apply a threshold to create a blob on the subject
            _, thresh = cv.threshold(blurred, 65, 255, cv.THRESH_BINARY_INV)

            # The dilation will be applied to ensure all parts of the screw remains
            dilated = cv.dilate(thresh, kernel, iterations=2)

            # Using contours, the blobs are detected
            contours, _ = cv.findContours(dilated, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

            # Crop size
            crop_size = 256
            half_crop = crop_size // 2

            for i, contour in enumerate(contours):
                x, y, w, h = cv.boundingRect(contour)

                # Calculate the midpoint of the blob
                center_x = x + w // 2
                center_y = y + h // 2

                # Midpoint centering
                x_start = max(center_x - half_crop, 0)
                y_start = max(center_y - half_crop, 0)

                # Ensure the image boundaries aren't exceeded
                x_end = min(x_start + crop_size, img.shape[1])
                y_end = min(y_start + crop_size, img.shape[0])
                x_start = max(x_end - crop_size, 0)
                y_start = max(y_end - crop_size, 0)

                # Crop
                cropped = img[y_start:y_end, x_start:x_end]

                # Save the cropped image
                cropped_filename = f"{filename.split('.')[0]}_crop_{i}.png"
                
                if random.random() < 0.8:
                    cropped_path = os.path.join(dataset_train, cropped_filename)
                else:
                    cropped_path = os.path.join(dataset_val, cropped_filename)
            
                cv.imwrite(cropped_path, cropped)